# Content-Based Filtering Recommendation System with Clustering 

This is the version without the clustering operations, everything else is the same. The other version is the main work of this project - this file is only used for comparision purpose.


In [1]:
# import libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity as cosine
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

np.set_printoptions(threshold=np.inf)

## Setup the dataset

In [2]:
# the dataset is accessed from https://www.kaggle.com/datasets/nikdavis/steam-store-games

chunks = pd.read_csv("data/steam.csv", chunksize=1024, index_col=0)
games = pd.DataFrame()

for chunk in chunks:
    games = pd.concat([games, chunk])
games

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
appid,,,,,,,,,,,,,,,,,
10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09
1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69
1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99


In [3]:
# the data is too large and my laptop was having memory space issues, so we only use the first 4000 rows for this project
games = games.head(4000)

# drop columns that are not needed to save memory space
games = games.drop(['english', 'developer', 'required_age',
                   'median_playtime', 'owners'], axis=1)

# only keep the year of the release date
games['release_year'] = games['release_date'].str.split('-').str[0]
games['release_year'] = games['release_year'].astype(int)

games.drop(columns=['release_date'], inplace=True)
games = games.dropna()  # drop rows with missing values

# from the count we know that 1 game has missing values in any of the columns we are interested in
games.describe()

,achievements,positive_ratings,negative_ratings,average_playtime,price,release_year
count,"3,999.00","3,999.00","3,999.00","3,999.00","3,999.00","3,999.00"
mean,23.51,"4,585.21",771.84,427.90,8.30,"2,012.71"
std,60.78,"47,884.81","7,581.66","2,580.30",7.07,2.65
min,0.00,0.00,0.00,0.00,0.00,"1,997.00"
25%,0.00,71.00,31.00,0.00,3.99,"2,011.00"
50%,12.00,280.00,93.00,24.00,6.99,"2,014.00"
75%,31.00,"1,345.50",303.00,256.00,10.99,"2,014.00"
max,"1,746.00","2,644,404.00","402,313.00","95,245.00",95.99,"2,019.00"


## Encode the textual features first


In [4]:
# check the dara types of the columns before encoding
print(games.dtypes)

name                 object
publisher            object
platforms            object
categories           object
genres               object
steamspy_tags        object
achievements          int64
positive_ratings      int64
negative_ratings      int64
average_playtime      int64
price               float64
release_year          int32
dtype: object


Before heading to the encoding steps, let's have a check on the numbers of the unique values:

In [5]:
columns_to_inspect = ['publisher', 'platforms',
                      'categories', 'genres', 'steamspy_tags']

# use a copy of the data here to avoid modifying the original data
df_check_only = games.copy()

# split the strings in the columns by the semicolon and then explode them into individual rows
for column in columns_to_inspect:
    df_check_only[column] = df_check_only[column].str.split(';')
    df_check_only = df_check_only.explode(column)

# calculate the number of unique values in each column
unique_values_counts = df_check_only[columns_to_inspect].nunique()
print(unique_values_counts)

publisher        1623
platforms           3
categories         29
genres             26
steamspy_tags     272
dtype: int64


Since there are too many unique values for 'publisher' and 'steamspy_tags' columns, using one hot encoding will lead to 'the curse of dimensionality' and increase computation, so other methods are used to deal with it subsequently.

Let's set up a function that splits and one-hot encodes multi-element columns, and call it to the columns that need to be applied in turn.

In [6]:
# the function takes the dataframe, the columns to split, the separator and the prefix for the new columns
def multi_one_hot(df, column_to_split, separate_by, col_prefix):

    # split the column by the separator
    df[column_to_split] = df[column_to_split].str.split(separate_by)

    # explode the column to have one element per row
    df_exploded = df.explode(column_to_split)

    # one-hot encode the columns
    df_encoded = pd.get_dummies(df_exploded, columns=[column_to_split])

    # get the names of the columns that were added
    added_columns = []
    for col in df_encoded.columns:
        if col.startswith(col_prefix):
            added_columns.append(col)

    # sum only the one-hot encoded columns and group by game name
    grouped = df_encoded.groupby('name')[added_columns].sum()

    # reset the index and set the game name as the index
    grouped = grouped.reset_index()
    grouped.set_index('name', inplace=True)

    return grouped


# save the one-hot encoded columns in a list
encoded_multi_columns = []

In [7]:
multi_columns_to_encode = ['platforms', 'categories', 'genres']

for column in multi_columns_to_encode:
    # copy the dataframe to make sure everytime the fucntion only works on the original data
    encoded_multi_column = multi_one_hot(
        games.copy(), column, ';', column + '_')
    encoded_multi_columns.append(encoded_multi_column)

In [8]:
one_hot_df = pd.concat(encoded_multi_columns, axis=1)

one_hot_df.set_index(games['name'], inplace=True)
one_hot_df

,platforms_linux,platforms_mac,platforms_windows,categories_Captions available,categories_Co-op,categories_Commentary available,categories_Cross-Platform Multiplayer,categories_Full controller support,categories_In-App Purchases,categories_Includes Source SDK,...,genres_Racing,genres_Sexual Content,genres_Simulation,genres_Software Training,genres_Sports,genres_Strategy,genres_Utilities,genres_Video Production,genres_Violent,genres_Web Publishing
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Team Fortress Classic,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Day of Defeat,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Deathmatch Classic,0,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Half-Life: Opposing Force,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FEIST,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Gloria Victis,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Out of Reach,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Since the 'publisher' column contains many unique values, while the frequency of the values maymatters (the size of the game publisher may be an important feature for game recommendations), the 'importance' of the word can be calculated by using TFIDF encoding.

In [9]:
copy1 = games.copy()
columns_to_tfidf = ['publisher']

# initiate an empty DataFrame to store all the encoded columns
tfidf_df = pd.DataFrame(index=one_hot_df.index)

# do TF-IDF encoding to the column(s) and combine them into one daraframe
for column in columns_to_tfidf:
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(copy1[column])
    tfidf_df_temp = pd.DataFrame(tfidf_matrix.toarray(
    ), columns=tfidf.get_feature_names_out(), index=one_hot_df.index)
    tfidf_df = pd.concat([tfidf_df, tfidf_df_temp], axis=1)

tfidf_df

,101,10tons,11,17,1c,244,2d,2dengine,2k,2play,...,zero,zeroscale,zojoi,zombie,zoo,zooloretto,zooptek,zotnip,zueira,zut
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Team Fortress Classic,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Day of Defeat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Deathmatch Classic,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Half-Life: Opposing Force,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FEIST,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Gloria Victis,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Out of Reach,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [10]:
# there are many new features after encoding - let's check the sparsity of the data
total_cells_tfidf = tfidf_df.size
non_zero_tfidf = tfidf_df.astype(bool).sum().sum()
percentage_non_zero_tfidf = (non_zero_tfidf / total_cells_tfidf) * 100
print(percentage_non_zero_tfidf, '%')

0.11446376238411068 %


In [11]:
# Very low sparsity, use PCA to downscale the data to keep only 95% of the variance
# https://stackoverflow.com/questions/76899048/how-to-obtain-specific-principal-components-from-pca-using-sklearn-or-matplotlib
pca = PCA(n_components=0.95)
tfidf_reduced = pca.fit_transform(tfidf_df)
tfidf_df = pd.DataFrame(tfidf_reduced, index=one_hot_df.index)

tfidf_df

,0,1,2,3,4,5,6,7,8,9,...,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,-0.00,-0.00,-0.03,0.00,-0.02,0.01,0.03,0.04,-0.01,0.07,...,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00
Team Fortress Classic,-0.00,-0.00,-0.03,0.00,-0.02,0.01,0.03,0.04,-0.01,0.07,...,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00
Day of Defeat,-0.00,-0.00,-0.03,0.00,-0.02,0.01,0.03,0.04,-0.01,0.07,...,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00
Deathmatch Classic,-0.00,-0.00,-0.03,0.00,-0.02,0.01,0.03,0.04,-0.01,0.07,...,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00
Half-Life: Opposing Force,-0.00,-0.00,-0.03,0.00,-0.02,0.01,0.03,0.04,-0.01,0.07,...,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FEIST,-0.00,-0.00,-0.02,0.00,-0.01,0.00,0.02,0.02,-0.01,0.03,...,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00
Gloria Victis,-0.01,-0.02,-0.11,-0.01,-0.11,-0.17,-0.07,-0.06,-0.02,-0.05,...,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00
Out of Reach,-0.01,-0.03,-0.01,-0.00,-0.02,0.01,0.08,0.24,-0.01,-0.06,...,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00


For the 'steamspy_tags' column, since there is no comparative relationship (e.g. 'action' and 'shoot' are only used as game tags, there is no logical relationship between them) for the words, the Bag of Words method is good enough to encode them.

In [12]:
copy2 = games.copy()
columns_to_bow = ['steamspy_tags']

bow_df = pd.DataFrame(index=one_hot_df.index)

for column in columns_to_bow:
    copy2[column] = copy2[column].str.replace(';', ' ')
    count = CountVectorizer()
    count_matrix = count.fit_transform(copy2[column])
    bow_df_temp = pd.DataFrame(count_matrix.toarray(
    ), columns=count.get_feature_names_out(), index=one_hot_df.index)
    bow_df = pd.concat([bow_df, bow_df_temp], axis=1)

bow_df

,1980s,2d,3d,40k,4x,6dof,access,action,adventure,agriculture,...,wars,web,werewolves,western,with,workshop,world,wrestling,your,zombies
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Team Fortress Classic,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Day of Defeat,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Deathmatch Classic,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Half-Life: Opposing Force,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FEIST,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Gloria Victis,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Out of Reach,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
# check the sparsity of the data
total_cells_bow = bow_df.size
non_zero_bow = bow_df.astype(bool).sum().sum()
percentage_non_zero_bow = (non_zero_bow / total_cells_bow) * 100
print(percentage_non_zero_bow, '%')

1.1078856670689412 %


In [14]:
# since the sparsity for the BoW data is also low, we will do PCA to it as well
pca = PCA(n_components=0.95)
bow_reduced = pca.fit_transform(bow_df)
bow_df = pd.DataFrame(bow_reduced, index=one_hot_df.index)

bow_df

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,-0.31,0.87,-0.46,-0.07,-0.13,0.02,0.30,0.10,-0.21,0.02,...,-0.00,-0.01,-0.00,-0.01,0.02,0.00,0.02,-0.00,0.01,0.01
Team Fortress Classic,-0.31,0.87,-0.46,-0.07,-0.13,0.02,0.30,0.10,-0.21,0.02,...,-0.00,-0.01,-0.00,-0.01,0.02,0.00,0.02,-0.00,0.01,0.01
Day of Defeat,0.31,0.32,-0.53,-0.36,0.08,-0.28,0.14,1.19,0.60,0.24,...,-0.00,-0.01,-0.00,-0.10,-0.08,-0.01,0.02,0.02,0.01,0.02
Deathmatch Classic,-0.31,0.87,-0.46,-0.07,-0.13,0.02,0.30,0.10,-0.21,0.02,...,-0.00,-0.01,-0.00,-0.01,0.02,0.00,0.02,-0.00,0.01,0.01
Half-Life: Opposing Force,-0.30,0.86,-0.42,0.09,-0.29,0.13,0.04,0.13,-0.40,-0.08,...,0.00,-0.00,-0.00,0.01,-0.00,-0.01,0.01,0.00,0.00,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FEIST,-0.74,0.45,0.59,0.08,-0.06,-0.14,0.06,0.11,-0.14,-0.09,...,-0.02,0.01,-0.01,0.00,0.00,-0.00,0.00,-0.00,0.01,-0.00
Gloria Victis,0.23,0.08,-0.39,-0.42,0.30,-0.25,0.26,0.23,-0.10,-0.28,...,-0.02,0.01,-0.05,0.00,0.01,0.00,-0.03,-0.01,0.01,-0.02
Out of Reach,0.25,0.10,-0.44,-0.33,0.14,-0.17,-0.21,1.06,0.57,0.05,...,0.01,0.01,0.02,0.01,-0.03,-0.02,0.01,0.04,0.01,0.01


## Get all the numerical features

We need each film to be represented by a vector of numbers.

The easiest way to start is to just pick the features that are already numerical!


In [15]:
n_columns = ['release_year', 'achievements', 'positive_ratings',
             'negative_ratings', 'average_playtime', 'price']

subset_features = games[n_columns].dropna()
subset_features.describe()

,release_year,achievements,positive_ratings,negative_ratings,average_playtime,price
count,"3,999.00","3,999.00","3,999.00","3,999.00","3,999.00","3,999.00"
mean,"2,012.71",23.51,"4,585.21",771.84,427.90,8.30
std,2.65,60.78,"47,884.81","7,581.66","2,580.30",7.07
min,"1,997.00",0.00,0.00,0.00,0.00,0.00
25%,"2,011.00",0.00,71.00,31.00,0.00,3.99
50%,"2,014.00",12.00,280.00,93.00,24.00,6.99
75%,"2,014.00",31.00,"1,345.50",303.00,256.00,10.99
max,"2,019.00","1,746.00","2,644,404.00","402,313.00","95,245.00",95.99


In [16]:
# standardise and fit the data
scaled_features = StandardScaler().fit_transform(subset_features)

In [17]:
num_features_df = pd.DataFrame(
    scaled_features, columns=n_columns, index=one_hot_df.index)
num_features_df

,release_year,achievements,positive_ratings,negative_ratings,average_playtime,price
name,,,,,,
Counter-Strike,-4.80,-0.39,2.51,0.34,6.66,-0.16
Team Fortress Classic,-5.18,-0.39,-0.03,-0.02,-0.06,-0.61
Day of Defeat,-3.67,-0.39,-0.02,-0.05,-0.09,-0.61
Deathmatch Classic,-4.42,-0.39,-0.07,-0.07,-0.07,-0.61
Half-Life: Opposing Force,-5.18,-0.39,0.01,-0.06,0.08,-0.61
...,...,...,...,...,...,...
FEIST,0.87,0.27,-0.09,-0.09,-0.14,-0.19
Gloria Victis,1.24,0.37,-0.04,0.06,-0.02,0.95
Out of Reach,2.00,0.34,-0.06,0.02,0.17,0.38


## Combine Features

Now we have all the features encoded and ready to combine them as the final features matrix for the recommendation system! 

The dataset was processed to a 3999 rows x 1329 columns shape, containing only the encoded features, corresponding to 3999 games, 1329 features: **numeric features + one-hot encoded features + TF-IDF encoded features + BoW encoded features (no clustering labels in this one)**, minus the original columns that had been replaced. The game names are set as the indexes for reference.

In [18]:
# set the original dataframe to have the game name as the indices
games.set_index(one_hot_df.index, inplace=True)

# combine all the dataframes into one final dataframe
final_df = pd.concat([num_features_df,
                      one_hot_df,
                      tfidf_df,
                      bow_df], axis=1)

final_df

,release_year,achievements,positive_ratings,negative_ratings,average_playtime,price,platforms_linux,platforms_mac,platforms_windows,categories_Captions available,...,93,94,95,96,97,98,99,100,101,102
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,-4.80,-0.39,2.51,0.34,6.66,-0.16,0,0,1,0,...,-0.00,-0.01,-0.00,-0.01,0.02,0.00,0.02,-0.00,0.01,0.01
Team Fortress Classic,-5.18,-0.39,-0.03,-0.02,-0.06,-0.61,0,0,1,0,...,-0.00,-0.01,-0.00,-0.01,0.02,0.00,0.02,-0.00,0.01,0.01
Day of Defeat,-3.67,-0.39,-0.02,-0.05,-0.09,-0.61,0,0,1,0,...,-0.00,-0.01,-0.00,-0.10,-0.08,-0.01,0.02,0.02,0.01,0.02
Deathmatch Classic,-4.42,-0.39,-0.07,-0.07,-0.07,-0.61,0,1,1,0,...,-0.00,-0.01,-0.00,-0.01,0.02,0.00,0.02,-0.00,0.01,0.01
Half-Life: Opposing Force,-5.18,-0.39,0.01,-0.06,0.08,-0.61,0,1,1,0,...,0.00,-0.00,-0.00,0.01,-0.00,-0.01,0.01,0.00,0.00,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FEIST,0.87,0.27,-0.09,-0.09,-0.14,-0.19,0,0,1,0,...,-0.02,0.01,-0.01,0.00,0.00,-0.00,0.00,-0.00,0.01,-0.00
Gloria Victis,1.24,0.37,-0.04,0.06,-0.02,0.95,1,1,1,0,...,-0.02,0.01,-0.05,0.00,0.01,0.00,-0.03,-0.01,0.01,-0.02
Out of Reach,2.00,0.34,-0.06,0.02,0.17,0.38,0,0,1,0,...,0.01,0.01,0.02,0.01,-0.03,-0.02,0.01,0.04,0.01,0.01


## Get to Similarities

In [19]:
# calculate the similarities between the games based on the cosine similarity
similarities = cosine(final_df)

similarities_df = pd.DataFrame(
    similarities, columns=games['name'], index=games['name'])
similarities_df

name,Counter-Strike,Team Fortress Classic,Day of Defeat,Deathmatch Classic,Half-Life: Opposing Force,Ricochet,Half-Life,Counter-Strike: Condition Zero,Half-Life: Blue Shift,Half-Life 2,...,Killing Floor - Toy Master,Deep Eclipse: New Space Odyssey,Infinite Game Works Episode 1,Motorama,Worms W.M.D,FEIST,Gloria Victis,Out of Reach,Tilt Brush,Deadstone
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,1.00,0.54,0.50,0.49,0.53,0.54,0.58,0.55,0.49,0.54,...,-0.02,-0.05,-0.10,-0.04,-0.05,-0.04,-0.07,-0.18,-0.11,-0.05
Team Fortress Classic,0.54,1.00,0.85,0.88,0.89,0.90,0.87,0.84,0.83,0.80,...,0.05,0.06,-0.08,0.11,0.03,0.17,-0.05,-0.28,-0.11,0.03
Day of Defeat,0.50,0.85,1.00,0.80,0.81,0.85,0.78,0.82,0.74,0.72,...,0.12,0.08,-0.07,0.08,-0.05,0.08,-0.06,-0.17,-0.08,0.04
Deathmatch Classic,0.49,0.88,0.80,1.00,0.89,0.87,0.86,0.86,0.80,0.78,...,0.08,0.09,-0.05,0.19,0.03,0.12,0.07,-0.29,-0.08,0.01
Half-Life: Opposing Force,0.53,0.89,0.81,0.89,1.00,0.89,0.88,0.83,0.92,0.88,...,0.05,0.06,-0.08,0.15,-0.00,0.12,-0.01,-0.28,-0.11,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FEIST,-0.04,0.17,0.08,0.12,0.12,0.14,0.05,0.02,0.07,0.23,...,0.17,0.42,0.45,0.45,0.53,1.00,0.37,0.28,0.31,0.35
Gloria Victis,-0.07,-0.05,-0.06,0.07,-0.01,0.01,-0.01,0.03,0.03,0.12,...,0.17,0.13,0.42,0.44,0.51,0.37,1.00,0.23,0.33,0.12
Out of Reach,-0.18,-0.28,-0.17,-0.29,-0.28,-0.23,-0.29,-0.31,-0.24,-0.17,...,0.10,0.18,0.17,0.21,0.33,0.28,0.23,1.00,0.46,0.48


In [20]:
# get the top 10 most similar games to our given game
game = "Counter-Strike"
n = 10

# sort the similarities of the given game
sorted = similarities_df.sort_values(by=game, ascending=False)

# get top n (excluding index 1 because thats the exact match / same film)
top_n = (sorted[game].index[1:n+1].values, (sorted[game].values[1:n+1])*100)
# since we have two 2x10 arrays, we need to transpose them to get the correct shape we desired
top_n_df = pd.DataFrame(top_n, index=['Game', 'Similarity(%)']).T
top_n_df

,Game,Similarity(%)
0,Counter-Strike: Source,82.32
1,The Banner Saga: Factions,73.58
2,Garry's Mod,72.45
3,FINAL FANTASY XIV Online,71.11
4,Darkstone,70.97
5,Arma 3,70.22
6,Fantasy Grounds,69.64
7,Shroud of the Avatar: Forsaken Virtues,68.90
8,Heroine's Quest: The Herald of Ragnarok,68.16
9,Arma 2: Operation Arrowhead,67.80


Let's pick the 4th most similar game to the game given, are they actually similar, if we look at the profiles?

In [21]:
games.loc[top_n_df.loc[3][0]]

C:\Users\tobys\AppData\Local\Temp\ipykernel_28300\2118746764.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  games.loc[top_n_df.loc[3][0]]


name                                         FINAL FANTASY XIV Online
publisher                                                 Square Enix
platforms                                                     windows
categories          Single-player;Multi-player;Online Multi-Player...
genres                                      Massively Multiplayer;RPG
steamspy_tags                        MMORPG;Massively Multiplayer;RPG
achievements                                                        0
positive_ratings                                                 9808
negative_ratings                                                 2107
average_playtime                                                28897
price                                                            9.99
release_year                                                     2014
Name: FINAL FANTASY XIV Online, dtype: object

In [22]:
games.loc[game]

name                                                   Counter-Strike
publisher                                                       Valve
platforms                                           windows;mac;linux
categories          Multi-player;Online Multi-Player;Local Multi-P...
genres                                                         Action
steamspy_tags                                  Action;FPS;Multiplayer
achievements                                                        0
positive_ratings                                               124534
negative_ratings                                                 3339
average_playtime                                                17612
price                                                            7.19
release_year                                                     2000
Name: Counter-Strike, dtype: object